In [1]:
import os
import sys
import math
import psutil
os.environ["_CCTRACE_"]="ON"
import cloudComPy as cc
from gendata import getSampleCloud, dataDir
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import laspy
print("Main : Libs loaded")

Main : Libs loaded


In [2]:
# TODO Load the data (using laspy -> cloudComPy)
# TODO optional convert into pandas dataframe (pandas limit to 100mb?)
# TODO Test functions:
# - getPointCloud
# - getSphere
# 
# TODO Slice the data
# TODO Subtract the data, need to ease the surface with gaussian fucntion, search the net for examples
# TODO Visualize the data
""" Current plan:
        - I cannot see what the functions from cloudComPy are doing(No peek definition), there is only documentation on webstite with no exapmles of functions alone.
                There are test cases, however they are not commented or documented on how they are achieving the results.
                And for now I don't see clear solution there on how to obtain the difference between two point clouds
        - I am creating my own point-clouds for the tests, I need to know how to create them (The libary accepts data in very specific way).
                I have made tests for reading the data from a file, but neither .las nor .LiData files are supported in a native way
                    Solution to that is to use additional libary called "laspy" which is a wrapper for las file format.(And then the data is in diffrent format than the normal data from cloudCompPy)
                Reason being that I need small sets of data and visualize them to see if the algorythms work.
                I want to see how it is visiualized and how it is subtracted on e.g. two point-cloud spheres. (I want to see if the difference is correct)
                If it works on small dataset it will also work on larger dataset.
                I cannot use the whole dataset with the mountain because it is too big.( No clear indication of the difference will be visible at this point in time.)
        - It requires theory/heavy algebra to understand how the algorythms work. Octree, Hausdorff distance, scalar fields etc.
                Even if these functions are present in libary the data must be prepared accordingly to use them. Even more so if I will use my own point cloud data.
        - Steps:
            1 Create a point-cloud spheres with the data
            2 Make a mesh/field form cloud-points(?) (The best way would be to calculate the the distance on the point-cloud and then use the distances to create scalar field, it would give the colours to the diffrenece)
            3 Use the Hausdorff distance algo to calculate the difference between the two spheres
            4 If it returns the point-cloud with the difference, then it is done.
            5 Apply to a big dataset
        - I might be missing something from the documentation that could do the job much faster, but I am not an expert on point-clouds.
        - Any suggestions/remarks are welcome
        
"""
""" Plan with Emanuele:
        1       Read the point cloud with Laspy
        2       Convert the point cloud to a cloudComPy format (A text file with three float numbers per line)
        3       Read the file with cloudComPy
        4       Create a mesh from the point cloud / repeat for the second point cloud
        5       Make a boolean INTERSECTION operation of the two meshes
        5.1     Calculate the Hausdorff distance between the two meshes (using cloudComPy) (Maybe use distance function form cloudComPy)
        6       Visualize the difference between the two meshes as one mesh
        7       Convert the mesh to a verticies only
        8       Convert the verticies to a point cloud and save it as a file

    Problem:
        1       The Hausdorff distance is not working properly. (it might not give the output that we desire)
        2       The information to which point-cloud does a point(Verticies) belong to is lost.
        2.1     It is possible to compare XYZ of a point from the difference Mesh of two point-clouds to points of the said clouds.
        2.2     But it about 70_000 points per cloud right now so that gives a lot of overhead.
        2.3     Need to find a solution for "height" of the difference mesh.

"""

' Plan with Emanuele:\n        1       Read the point cloud with Laspy\n        2       Convert the point cloud to a cloudComPy format (A text file with three float numbers per line)\n        3       Read the file with cloudComPy\n        4       Create a mesh from the point cloud / repeat for the second point cloud\n        5       Make a boolean INTERSECTION operation of the two meshes\n        5.1     Calculate the Hausdorff distance between the two meshes (using cloudComPy) (Maybe use distance function form cloudComPy)\n        6       Visualize the difference between the two meshes as one mesh\n        7       Convert the mesh to a verticies only\n        8       Convert the verticies to a point cloud and save it as a file\n\n    Problem:\n        1       The Hausdorff distance is not working properly. (it might not give the output that we desire)\n        2       The information to which point-cloud does a point(Verticies) belong to is lost.\n        2.1     It is possible to com

In [3]:
path1 = r"C:\\Users\\szinp\\Desktop\\lazorScan_project\\testData\\cloud_e4d00e9_CutResult_17_7_42.LiData"
path2 = r"C:\\Users\\szinp\\Desktop\\lazorScan_project\\testData\\cloud_e4d00e9_CutResult_17_14_7.las"
path3 = r"C:\\Users\\szinp\\Desktop\\lazorScan_project\\testData\\cloud_e4d00e9_CutResult_17_14_7.LiData"
path4 = r"C:\\Users\\szinp\\Desktop\\lazorScan_project\\testData\\laser_1_2021-06-01-09-54-35_color_CutResult_17_14_7.las"
path5 = r"C:\\Users\\szinp\\Desktop\\lazorScan_project\\testData\\laser_1_2021-06-01-09-54-35_color_CutResult_17_14_7.LiData"
assert os.path.isfile(path1)
assert os.path.isfile(path2)
assert os.path.isfile(path3)
assert os.path.isfile(path4)
assert os.path.isfile(path5)
# with open(file=path, mode="r") as cloudFile:
#     print("Main : File opened")

In [4]:
#* os.environ["_CCTRACE_"]="ON"                                           # only if you want debug traces from C++

#* import cloudComPy as cc                                                # import the CloudComPy module

#* cloud = cc.loadPointCloud("myCloud.xyz")                               # read a point cloud from a file
#* print("cloud name: %s"%cloud.getName())

#* res=cc.computeCurvature(cc.CurvatureType.GAUSSIAN_CURV, 0.05, [cloud]) # compute curvature as a scalar field
#* nsf = cloud.getNumberOfScalarFields()
#* sfCurv=cloud.getScalarField(nsf-1)
#* cloud.setCurrentOutScalarField(nsf-1)
#* filteredCloud=cc.filterBySFValue(0.01, sfCurv.getMax(), cloud)         # keep only the points above a given curvature

#* ok = filteredCloud.exportCoordToSF(False, False, True)                 # Z coordinate as a scalar Field
#* nsf = cloud.getNumberOfScalarFields()
#* sf1=filteredCloud.getScalarField(nsf-1)
#* mean, var = sf1.computeMeanAndVariance()

#* # using Numpy...

#* coordinates = filteredCloud.toNpArrayCopy()                            # coordinates as a numpy array
#* x=coordinates[:,0]                                                     # x column
#* y=coordinates[:,1]
#* z=coordinates[:,2]

#* f=(2*x-y)*(x+3*y)                                                      # elementwise operation on arrays

#* asf1=sf1.toNpArray()                                                   # scalar field as a numpy array
#* sf1.fromNpArrayCopy(f)                                                 # replace scalar field values by a numpy array

#* res=cc.SavePointCloud(filteredCloud,"myModifiedCloud.bin")             #save the point cloud to a file

### Reading the first .LAS file
[]: # Language: python
[]: # Path: substractionPointCloudTest.ipynb

In [5]:
# Using path2 as input (This is .las data file)
cc.initCC()

# listCoord_XYZ = np.array([], ndmin = 2)
listCoord_XYZ = np.empty([1,3])
x:float = 0.0
y:float = 0.0
z:float = 0.0
#! remark - path 2 has a lot of points, so it takes a while to read it (76_568_359 to be exact).
with laspy.open(source = path2) as cloudFileHeader:
    if type(cloudFileHeader) != None:
        cloudFile = laspy.read(source=path2)
        try:
            print("Cloud loaded")
            print(f"cloud name:{cloudFile.header}")
        except:
            print(f"cloud name:<unknown>")
    # limit = 0
    #? It seems that chunk_iterator helps to read data in batches. In every batch it reads the data and then it moves to the next batch.
    #? Points.array is in fact a one point array. if we set chunk_iterator is set to (1)
    #? What is the data inside of the point? It must be x,y,z and something else. It is possible to extract x,y,z from the point. Is it enough data?
    for points in cloudFileHeader.chunk_iterator(1):
        # print("type(points)\n", type(points))
        # print("points _>\n", points)
        # print("points array _>\n", points.array)
        # print("type(points.array[0]) _>\n", type(points.array[0]))
        # print("points.array[0] _>\n", points.array[0])
        # print("points.array[0]['X'] _>\n", points.array[0]['X'])

        # print("points.dtype\n", points.dtype)
        # print("points.dtype.names\n", points.dtype.names)
        x,y,z = points.array[0]['X'], points.array[0]['Y'], points.array[0]['Z']
        valuesToAppend = np.array([[x,y,z]])
        # print("(valuesToAppend).shape",(valuesToAppend).shape)
        # print( "listCoord_XYZ.shape", listCoord_XYZ.shape)

        listCoord_XYZ = np.append(arr = listCoord_XYZ, values= valuesToAppend , axis=0)
        

        # if limit == 10:
        #     break
        # limit += 1
    print("listCoord_XYZ\n", listCoord_XYZ)

Cloud loaded
cloud name:<LasHeader(1.4, <PointFormat(7, 0 bytes of extra dims)>)>


KeyboardInterrupt: 

### Saving the first result in a .txt file for compatibility with CloudComPy
[]: # Language: python
[]: # Path: substractionPointCloudTest.ipynb

In [ ]:
np.savetxt("listCoord_XYZ_1.txt", listCoord_XYZ, delimiter=" ", newline="\n")

### Reading the second .LAS file 
[]: # Language: python
[]: # Path: substractionPointCloudTest.ipynb

In [ ]:
listCoord_XYZ = np.empty([1,3])
x:float = 0.0
y:float = 0.0
z:float = 0.0
with laspy.open(source = path4) as cloudFileHeader:
    if type(cloudFileHeader) != None:
        cloudFile = laspy.read(source=path4)
        try:
            print("Cloud loaded")
            print(f"cloud name:{cloudFile.header}")
        except:
            print(f"cloud name:<unknown>")
    # limit = 0
    #? It seems that chunk_iterator helps to read data in batches. In every batch it reads the data and then it moves to the next batch.
    #? Points.array is in fact a one point array. if we set chunk_iterator is set to (1)
    #? What is the data inside of the point? It must be x,y,z and something else. It is possible to extract x,y,z from the point. Is it enough data?
    for points in cloudFileHeader.chunk_iterator(1):
        # print("type(points)\n", type(points))
        # print("points _>\n", points)
        # print("points array _>\n", points.array)
        # print("type(points.array[0]) _>\n", type(points.array[0]))
        # print("points.array[0] _>\n", points.array[0])
        # print("points.array[0]['X'] _>\n", points.array[0]['X'])

        # print("points.dtype\n", points.dtype)
        # print("points.dtype.names\n", points.dtype.names)
        x,y,z = points.array[0]['X'], points.array[0]['Y'], points.array[0]['Z']
        valuesToAppend = np.array([[x,y,z]])
        # print("(valuesToAppend).shape",(valuesToAppend).shape)
        # print( "listCoord_XYZ.shape", listCoord_XYZ.shape)

        listCoord_XYZ = np.append(arr = listCoord_XYZ, values= valuesToAppend , axis=0)
        

        # if limit == 10:
        #     break
        # limit += 1
    print("listCoord_XYZ\n", listCoord_XYZ)

### Saving the second result in a .txt file for compatibility with CloudComPy
[]: # Language: python
[]: # Path: substractionPointCloudTest.ipynb

In [ ]:
np.savetxt("listCoord_XYZ_2.txt", listCoord_XYZ, delimiter=" ", newline="\n")

#### TODO: Make a progress bar for computing

In [ ]:
import time
import progressbar

widgets = [' [',
		progressbar.Timer(format= 'elapsed time: %(elapsed)s'),
		'] ',
		progressbar.Bar('*'),' (',
		progressbar.ETA(), ') ',
		]

bar = progressbar.ProgressBar(max_value=200,
							widgets=widgets).start()

for i in range(200):
	time.sleep(0.1)
	bar.update(i)
